In [1]:
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
import numpy as np

model = VGG16(weights='imagenet', include_top=False)
model.summary()

img_path = '1.png'
img = image.load_img(img_path, target_size=(224, 224))
img_data = image.img_to_array(img)
img_data = np.expand_dims(img_data, axis=0)
img_data = preprocess_input(img_data)

vgg16_feature = model.predict(img_data)

print(vgg16_feature.shape)

Using TensorFlow backend.


58892288/58889256 [==============================] - 286s 5us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_po

### Using K-means to cluster a set of images

In [11]:
from sklearn.cluster import KMeans

In [12]:
import os

vgg16_feature_list = []
directory = '../example_output'

for filename in os.listdir(directory):
    img_path = directory + '/' + filename
    
    img = image.load_img(img_path, target_size=(224, 224))
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocess_input(img_data)

    vgg16_feature = model.predict(img_data)
    vgg16_feature_np = np.array(vgg16_feature)
    vgg16_feature_list.append(vgg16_feature_np.flatten())
        
        
vgg16_feature_list_np = np.array(vgg16_feature_list)
kmeans = KMeans(n_clusters=2, random_state=0).fit(vgg16_feature_list_np)

In [37]:
vgg16_feature_list_np.shape

(1965, 25088)

In [14]:
kmeans.n_clusters

2

In [15]:
kmeans.predict(vgg16_feature_list_np)

array([0, 0, 0, ..., 1, 1, 0])

In [16]:
import collections
collections.Counter(kmeans.labels_)

Counter({0: 1257, 1: 708})

(1965,)

In [26]:
# define input and outputFolder
noisyFolder='NoisyData1/'
acceptedFolder = 'AcceptedData1/'

# Create directory
try:
    # Create target Directory
    os.mkdir(noisyFolder)
    print("Directory " , noisyFolder ,  " Created ") 
except FileExistsError:
    print("Directory " , noisyFolder ,  " already exists")
    

# Create directory
try:
    # Create target Directory
    os.mkdir(acceptedFolder)
    print("Directory " , noisyFolder ,  " Created ") 
except FileExistsError:
    print("Directory " , noisyFolder ,  " already exists")

Directory  NoisyData1/  Created 
Directory  NoisyData1/  Created 


In [27]:
from shutil import copyfile

filesNames = np.array(os.listdir(directory))
labels = kmeans.labels_

for i in range(0, kmeans.labels_.shape[0]):
    fileName = filesNames[i]
    label = labels[i]
    
    src = directory + '/' + fileName
    if label == 0:
        dst = acceptedFolder + fileName
    else:
        dst = noisyFolder + fileName
        
    copyfile(src, dst)
    

### Based on color values

In [43]:
%matplotlib inline
import matplotlib.pyplot as plt
from scipy import ndimage
from sklearn import cluster
import numpy as np
import cv2

feature_list = []
directory = '../example_output'

for filename in os.listdir(directory):
    img_path = directory + '/' + filename
    image = ndimage.imread(img_path)
    image = cv2.pyrMeanShiftFiltering(image, 30, 51)
    x, y, z = image.shape
    image_2d = image.reshape(x*y, z)
    feature_list.append(image_2d.flatten())
    

feature_list_np = np.array(feature_list)
feature_list_np.shape
#     plt.figure(figsize = (15,8))
#     plt.imshow(image)

C:\Users\sohil\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.
  del sys.path[0]


(1965, 1228800)

In [50]:
feature_list_np[577]

array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)

In [ ]:

image_2d.shape

In [46]:
kmeans_cluster = KMeans(n_clusters=3)
kmeans_cluster.fit(feature_list_np)
cluster_centers = kmeans_cluster.cluster_centers_
cluster_labels = kmeans_cluster.labels_

import collections
collections.Counter(kmeans_cluster.labels_)

MemoryError: 